# JupyterNotebookViewer Demo for Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/iory/scikit-robot/blob/main/examples/notebooks/colab_jupyter_viewer_demo.ipynb)

This notebook demonstrates scikit-robot's JupyterNotebookViewer in Google Colab.

## Install scikit-robot

First, we need to install scikit-robot and its dependencies.

In [ ]:
# Install scikit-robot
!pip install scikit-robot -q
print("✓ Installation complete!")

## Import Libraries

In [ ]:
import time

import numpy as np

import skrobot


print(f"scikit-robot version: {skrobot.__version__}")

## Multiple Robot Models Demo

First, let's display multiple robot models in a grid layout!

In [ ]:
# Create viewer for multiple robots
multi_viewer = skrobot.viewers.JupyterNotebookViewer(height=600)

# Create multiple robot models
robots = [
    skrobot.models.Kuka(),
    skrobot.models.Fetch(),
    skrobot.models.Nextage(),
    skrobot.models.PR2(),
    skrobot.models.Panda(),
]

# Calculate grid layout
def get_tile_shape(num, hw_ratio=1):
    r_num = int(round(np.sqrt(num / hw_ratio)))
    c_num = 0
    while r_num * c_num < num:
        c_num += 1
    while (r_num - 1) * c_num >= num:
        r_num -= 1
    return r_num, c_num

nrow, ncol = get_tile_shape(len(robots))
row_spacing, col_spacing = 2, 2

# Add robots in grid layout
for i in range(nrow):
    for j in range(ncol):
        try:
            robot = robots[i * ncol + j]
        except IndexError:
            break
        
        # Add platform under each robot
        platform = skrobot.model.Box(extents=(row_spacing - 0.01, col_spacing - 0.01, 0.01))
        platform.translate((row_spacing * i, col_spacing * j, -0.01))
        multi_viewer.add(platform)
        
        # Position robot
        robot.translate((row_spacing * i, col_spacing * j, 0))
        multi_viewer.add(robot)

multi_viewer.set_camera(angles=[np.deg2rad(30), 0, 0])

print(f"Created {len(robots)} robots in {nrow}x{ncol} grid:")
for robot in robots:
    print(f"  - {robot.__class__.__name__}")

multi_viewer.show()

## Grasp and Pull Box Demo

Now let's demonstrate a practical task: grasping and pulling a box.

In [ ]:
# Create PR2 robot for manipulation task
from skrobot.coordinates.geo import midcoords
from skrobot.model.primitives import Box

robot = skrobot.models.PR2()

# Define right arm link list
link_list = [
    robot.r_shoulder_pan_link,
    robot.r_shoulder_lift_link,
    robot.r_upper_arm_roll_link,
    robot.r_elbow_flex_link,
    robot.r_forearm_roll_link,
    robot.r_wrist_flex_link,
    robot.r_wrist_roll_link]

# Create end-effector coordinates
rarm_end_coords = skrobot.coordinates.CascadedCoords(
    parent=robot.r_gripper_tool_frame,
    name='rarm_end_coords')
move_target = rarm_end_coords

# Create viewer and add robot
grasp_viewer = skrobot.viewers.JupyterNotebookViewer(height=600)
grasp_viewer.add(robot)

# Create box
box = Box(extents=[0.1, 0.1, 0.2], with_sdf=True)
box_center = np.array([0.9, -0.2, 0.9])
box.translate(box_center)
grasp_viewer.add(box)

print("Robot and box created!")

### Execute Grasp and Pull Sequence

Watch the robot grasp and pull the box!

In [ ]:
# Display viewer in THIS cell
grasp_viewer.show()

# 1. Move to initial pose and open gripper
print("1. Moving to initial pose...")
robot.reset_pose()
target_coords = skrobot.coordinates.Coordinates([0.5, -0.3, 0.7], [0, 0, 0])
robot.inverse_kinematics(target_coords, link_list=link_list, move_target=move_target)
robot.gripper_distance(0.093, arm='rarm')  # Open gripper
grasp_viewer.redraw()
time.sleep(1)

# 2. Move to box
print("2. Moving to box...")
robot.look_at(box)
start_coords = move_target.copy_worldcoords()
target_coords = box.copy_worldcoords()

for i in range(20):
    robot.inverse_kinematics(
        midcoords(i / 20.0, start_coords, target_coords),
        link_list=link_list,
        move_target=move_target)
    grasp_viewer.redraw()
    time.sleep(0.1)

# 3. Grasp box
print("3. Grasping box...")
robot.gripper_distance(0.036, arm='rarm')  # Close gripper
move_target.assoc(box)  # Attach box to gripper
grasp_viewer.redraw()
time.sleep(1)

# 4. Pull box
print("4. Pulling box...")
target_coords = skrobot.coordinates.Coordinates([0.5, -0.3, 0.7])
start_coords = box

for i in range(20):
    robot.inverse_kinematics(
        midcoords(i / 20.0, start_coords, target_coords),
        link_list=link_list,
        move_target=move_target)
    robot.look_at(box)
    grasp_viewer.redraw()
    time.sleep(0.1)

print("\n✓ Grasp and pull complete!")

In [ ]:
html = grasp_viewer.to_html()

# Save to file
with open('/content/robot_grasp.html', 'w') as f:
    f.write(html)

print("Saved to: /content/robot_grasp.html")
print("You can download this file and open it in any browser!")

# Display download link
from google.colab import files

print("\nClick below to download:")
files.download('/content/robot_grasp.html')

## Summary

✓ JupyterNotebookViewer works in Google Colab!

Features demonstrated:
1. **Multiple robot models** (Kuka, Fetch, Nextage, PR2, Panda) in grid layout
   - Synchronized rotation animation
2. **Grasp and pull manipulation task**
   - Inverse kinematics for motion planning
   - Gripper control (open/close)
   - Object attachment to end-effector
   - Smooth interpolated motion

Technical features:
- 3D robot visualization with interactive camera controls (mouse)
- Smooth animation updates with `redraw()`
- No white flickering during updates
- HTML export for standalone viewing

Implementation:
- IPython.display.HTML for notebook integration
- iframe embedding for 3D rendering
- postMessage API for smooth updates without page reload
- three.js for WebGL-based 3D rendering

All features work perfectly in both Jupyter and Google Colab!